In [1]:
import os
import gc
import time
import tempfile
import warnings
import functools
warnings.filterwarnings(action='ignore')

import pandas as pd

from memory_profiler import memory_usage

%load_ext memory_profiler

In [2]:
pd.__version__

'0.24.2'

In [3]:
TMP_DIR = "./tmp"
os.makedirs(TMP_DIR, exist_ok=True)

SIZE_MAP = {
    "KB": 1024 ** 1,
    "MB": 1024 ** 2,
    "GB": 1024 ** 3,
}


def profile(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        start_time = time.time()
        initial_memory = memory_usage()[0]
        result = func(*args, **kwargs)
        peak_memory = max(memory_usage())
        increment_memory = peak_memory - initial_memory
        end_time = time.time()
        print(f"[{func.__name__}]\nElapsed Time: {end_time - start_time:,.3f} seconds,\nPeak Memory: {peak_memory*1.04858:,.2f} MB,\nIncrement Memory: {increment_memory*1.04858:,.2f} MB")
        return result
    return wrapper


def memory_usage_pdobj(pdobj, size="MB"):
    return pdobj.memory_usage(deep=True).sum() / SIZE_MAP.get(size, "MB")

In [4]:
@profile
def save_as(df: pd.DataFrame, file_format: str) -> str:
    temp_filename = tempfile.mktemp(dir=TMP_DIR)
    if file_format == "csv":
        df.to_csv(temp_filename, index=False)
    elif file_format == "pickle":
        df.to_pickle(temp_filename)
    elif file_format == "feather":
        df.to_feather(temp_filename)
    elif file_format == "hdf5":
        df.to_hdf(temp_filename, key='df', mode='w')
    elif file_format == "parquet":
        df.to_parquet(temp_filename)
    return temp_filename


@profile
def load_as(temp_filename: str, file_format: str) -> None:
    if file_format == "csv":
        df = pd.read_csv(temp_filename)
    elif file_format == "pickle":
        df = pd.read_pickle(temp_filename)
    elif file_format == "feather":
        df = pd.read_feather(temp_filename)
    elif file_format == "hdf5":
        df = pd.read_hdf(temp_filename, 'df')
    elif file_format == "parquet":
        df = pd.read_parquet(temp_filename)

    del(df)
    gc.collect()
    os.remove(temp_filename)


def get_filesize(filepath: str, size="MB"):
    return os.path.getsize(filepath) / SIZE_MAP.get(size, "MB")


In [5]:
GJ_FILEPATH = "../data/open-nhis-gj-all.csv"
T20_FILEPATH = "../data/open-nhis-t20-all.csv"

## 1. Memory Optimization

In [6]:
%%time
%%memit
gj_np = pd.read_csv(GJ_FILEPATH)

peak memory: 8940.03 MiB, increment: 8847.55 MiB
CPU times: user 31 s, sys: 2.37 s, total: 33.4 s
Wall time: 33.5 s


In [7]:
print(f"GJ Numpy: {memory_usage_pdobj(gj_np):,.2f} MB")

GJ Numpy: 10,442.86 MB


### 1.1 데이터 형식 변환

데이터 형식 변환에 따른 메모리 사용량

- 코드북 사용
- 파일을 읽을 때 데이터 타입 지정

(+) pyarrow 데이터 타입을 지정했을 때 변화

#### 코드북 사용

In [8]:
CODEBOOK = {
    "SEX": {1: "남자", 2: "여자"},
    "SIDO": {11: "서울특별시", 26: "부산광역시", 27: "대구광역시", 28: "인천광역시", 29: "광주광역시", 30: "대전광역시",
             31: "울산광역시", 36: "세종특별자치시", 42: "강원도", 43: "충청북도", 44: "충청남도", 45: "전라북도",
             46: "전라남도", 47: "경상북도", 48: "경상남도", 49: "제주특별자치도"},
    "HEAR_LEFT": {1: "정상", 2: "비정상"},
    "HEAR_RIGHT": {1: "정상", 2: "비정상"},
    "SMK_STAT_TYPE_CD": {1: "비흡연", 2: "금연", 3: "흡연"},
    "DRK_YN": {0: "N", 1: "Y"},
}

CODEBOOK = {col: {v: str(k) for k, v in d.items()} for col, d in CODEBOOK.items()}

In [9]:
%%time
%%memit
gj_np = gj_np.replace(CODEBOOK)

peak memory: 12562.86 MiB, increment: 3667.71 MiB
CPU times: user 14.3 s, sys: 996 ms, total: 15.3 s
Wall time: 15.4 s


In [10]:
print(f"GJ Numpy: {memory_usage_pdobj(gj_np):,.2f} MB") 
# print(f"GJ PyArrow: {memory_usage_pdobj(gj_pa):,.2f} MB") 

GJ Numpy: 8,005.85 MB


#### 데이터 타입 지정

In [11]:
GJ_USECOLS = ["HCHK_YEAR", "IDV_ID", "SEX", "AGE_GROUP", "SIDO", "HEIGHT", "WEIGHT", "SIGHT_LEFT", "SIGHT_RIGHT",
              "HEAR_LEFT", "HEAR_RIGHT", "BP_HIGH", "BP_LWST", "BLDS", "TOT_CHOLE", "HMG", "OLIG_PROTE_CD",
              "SGOT_AST", "SGPT_ALT", "GAMMA_GTP", "SMK_STAT_TYPE_CD", "DRK_YN"]

# 2.0부터는 data type으로 float8, float16 사용 못함
GJ_DTYPES_NP = {
    "HCHK_YEAR": "int16",
    "IDV_ID": "int32",
    "SEX": "category",
    "AGE_GROUP": "category",
    "SIDO": "category",
    "HEIGHT": "float32",
    "WEIGHT": "float32",
    "SIGHT_LEFT": "category",
    "SIGHT_RIGHT": "category",
    "HEAR_LEFT": "category",
    "HEAR_RIGHT": "category",
    "BP_HIGH": "float32",
    "BP_LWST": "float32",
    "BLDS": "float32",
    "TOT_CHOLE": "float32",
    "HMG": "float32",
    "OLIG_PROTE_CD": "float32",
    "SGOT_AST": "float32",
    "SGPT_ALT": "float32",
    "GAMMA_GTP": "float32",
    "SMK_STAT_TYPE_CD": "category",
    "DRK_YN": "category",
}

GJ_DTYPES_PA = {k: v + "[pyarrow]" if v.startswith(("int", "float")) else v for k, v in GJ_DTYPES_NP.items()}

In [12]:
%%time
%%memit
gj_np = pd.read_csv(GJ_FILEPATH, dtype=GJ_DTYPES_NP)

peak memory: 10406.35 MiB, increment: 820.91 MiB
CPU times: user 33.3 s, sys: 1.27 s, total: 34.6 s
Wall time: 34.7 s


In [13]:
print(f"GJ Numpy: {memory_usage_pdobj(gj_np):,.2f} MB")

GJ Numpy: 1,062.77 MB


### 1.2 파일 포맷에 따른 처리시간, 메모리사용량, 파일크기 비교
csv, pickle, feather, hdf5, parquet 포맷간 비교분석
- 읽기/쓰기 시간 비교
- IO 시에 메모리 사용량 비교
- 저장된 파일의 크기 비교

(+) pyarrow 데이터 타입을 지정했을 때 변화

In [14]:
gj_np = pd.read_csv(GJ_FILEPATH)

In [15]:
for file_format in ["csv", "pickle", "feather", "parquet"]:
    print(f"\n\n=== {file_format} ===")
    temp = save_as(gj_np, file_format)
    print(f"File size: {get_filesize(temp):,.2f} MB")
    load_as(temp, file_format)



=== csv ===
[save_as]
Elapsed Time: 271.642 seconds,
Peak Memory: 11,625.15 MB,
Increment Memory: 0.43 MB
File size: 2,175.33 MB
[load_as]
Elapsed Time: 36.749 seconds,
Peak Memory: 12,054.47 MB,
Increment Memory: 429.32 MB


=== pickle ===
[save_as]
Elapsed Time: 5.543 seconds,
Peak Memory: 12,054.53 MB,
Increment Memory: 0.06 MB
File size: 2,928.58 MB
[load_as]
Elapsed Time: 4.473 seconds,
Peak Memory: 11,574.78 MB,
Increment Memory: -479.75 MB


=== feather ===
[save_as]
Elapsed Time: 4.156 seconds,
Peak Memory: 11,792.04 MB,
Increment Memory: 217.26 MB
File size: 1,068.58 MB
[load_as]
Elapsed Time: 3.244 seconds,
Peak Memory: 15,239.13 MB,
Increment Memory: 3,447.09 MB


=== parquet ===
[save_as]
Elapsed Time: 12.099 seconds,
Peak Memory: 15,579.09 MB,
Increment Memory: 339.96 MB
File size: 301.21 MB
[load_as]
Elapsed Time: 3.160 seconds,
Peak Memory: 18,036.66 MB,
Increment Memory: 2,457.57 MB


## 2. Performance Optimization

In [16]:
import numpy as np
try:
    import pyarrow as pa
except:
    pass

### 2.1 건강위험지수 산출
numeric arithmetic operations의 성능
- iteration
- np vectorization
- pa vectorization

In [17]:
SCORING_COLS = ['IDV_ID', 'SEX', 'AGE_GROUP', 'HEIGHT', 'WEIGHT', 'BLDS', 'TOT_CHOLE',
                'HMG', 'OLIG_PROTE_CD', 'SGOT_AST', 'SGPT_ALT', 'SMK_STAT_TYPE_CD', 'DRK_YN']


def scoring_health_pd(patient):
    bmi = ((patient['WEIGHT'] / (patient['HEIGHT']/100)**2) >= 30) * 1  # 체질량(BMI)
    blds = (patient['BLDS'] >= 125) * 1  # 공복혈당 BLDS
    chole = (patient['TOT_CHOLE'] >= 130) * 1  # 총콜레스테롤 TOT_CHOLE
    hmg = (patient['HMG'] < 12) * 1  # 혈색소 HMG
    sg = ((patient['SGOT_AST'] >= 40) | (patient['SGPT_ALT'] >= 40)) * 1  # 혈청지오티 SGOT_AST, SGPT_ALT
    smoke = (patient['SMK_STAT_TYPE_CD'] == 3) * 1  # 흡연 SMOKE
    drink = (patient['DRK_YN'] == 1) * 1 # 음주 DRINK
    
    patient_score = np.sum([bmi, blds, chole, hmg, sg, smoke, drink], axis=0)
    return patient_score


def scoring_health_np(patient):
    bmi = ((patient['WEIGHT'].values / (patient['HEIGHT'].values/100)**2) >= 30) * 1  # 체질량(BMI)
    blds = (patient['BLDS'].values >= 125) * 1  # 공복혈당 BLDS
    chole = (patient['TOT_CHOLE'].values >= 130) * 1  # 총콜레스테롤 TOT_CHOLE
    hmg = (patient['HMG'].values < 12) * 1  # 혈색소 HMG
    sg = ((patient['SGOT_AST'].values >= 40) | (patient['SGPT_ALT'].values >= 40)) * 1  # 혈청지오티 SGOT_AST, SGPT_ALT
    smoke = (patient['SMK_STAT_TYPE_CD'].values == 3) * 1  # 흡연 SMOKE
    drink = (patient['DRK_YN'].values == 1) * 1 # 음주 DRINK
    
    patient_score = np.sum([bmi, blds, chole, hmg, sg, smoke, drink], axis=0)
    return patient_score

In [18]:
%%time
%%memit
gj_np = pd.read_csv(GJ_FILEPATH, usecols=SCORING_COLS, dtype=GJ_DTYPES_NP)
gj_np = gj_np.replace(CODEBOOK).fillna(method="bfill")

peak memory: 17514.19 MiB, increment: 313.14 MiB
CPU times: user 35.5 s, sys: 2.16 s, total: 37.7 s
Wall time: 37.9 s


#### Iteration using `iterrows()`

In [19]:
%%time
scores_itterrows = []
for _, patient in gj_np.iterrows():
    patient_score = scoring_health_pd(patient)
    scores_itterrows.append(patient_score)

CPU times: user 28min 9s, sys: 7.88 s, total: 28min 17s
Wall time: 28min 17s


#### Iteration using `.apply()`

In [20]:
%%time
%%memit
scores_apply = gj_np.apply(scoring_health_pd, axis=1)

peak memory: 27326.79 MiB, increment: 10059.64 MiB
CPU times: user 28min 39s, sys: 10.1 s, total: 28min 49s
Wall time: 28min 49s


#### Vectorization with `pd.Series`

In [21]:
%%time
%%memit
scores_pd = scoring_health_pd(gj_np)

peak memory: 20099.47 MiB, increment: 997.70 MiB
CPU times: user 2.13 s, sys: 516 ms, total: 2.64 s
Wall time: 2.77 s


#### Vectorization with `np.array`

In [22]:
%%time
%%memit
scores_np = scoring_health_np(gj_np)

peak memory: 20153.23 MiB, increment: 1052.08 MiB
CPU times: user 1.1 s, sys: 470 ms, total: 1.57 s
Wall time: 1.68 s


#### Python List, NumPy Array

In [23]:
%%time
%%memit
# Python List
weight_py = gj_np["WEIGHT"].tolist()
height_py = gj_np["HEIGHT"].tolist()

bmi_py = list(map(lambda a, b: a / (b/100)**2, weight_py, height_py))

peak memory: 19457.78 MiB, increment: 211.64 MiB
CPU times: user 3.02 s, sys: 469 ms, total: 3.49 s
Wall time: 3.61 s


In [24]:
%%time
%%memit
# Numpy Array
weight_np = gj_np["WEIGHT"].values
height_np = gj_np["HEIGHT"].values

bmi_np = np.divide(weight_np, np.power(np.divide(height_np, 100), 2))

peak memory: 19308.57 MiB, increment: 0.02 MiB
CPU times: user 650 ms, sys: 111 ms, total: 761 ms
Wall time: 873 ms


### 2.2 병원 비용이 높은 최상위 진료내역서 5개

In [25]:
T20_DTYPES_NP = {
    "STND_Y": "int16",
    "IDV_ID": "int32",
    "KEY_SEQ": "int32",
    "SEX": "category",
    "AGE_GROUP": "category",
    "SIDO": "category",
    "RECU_FR_DT": "object",
    "MAIN_SICK": "category",
    "SUB_SICK": "category",
    "VSCN": "int16",
    "EDEC_ADD_RT": "float32",
    "EDEC_TRAMT": "int32",
    "EDEC_SBRDN_AMT": "int32",
    "EDEC_JBRDN_AMT": "int32",
}

T20_DTYPES_PA = {k: v + "[pyarrow]" if v.startswith(("int", "float", "string")) else v for k, v in T20_DTYPES_NP.items()}

In [26]:
%%time
%%memit
t20_np = pd.read_csv(T20_FILEPATH, dtype=T20_DTYPES_NP)
t20_np = t20_np.replace(CODEBOOK)

peak memory: 34067.69 MiB, increment: 14975.31 MiB
CPU times: user 5min 17s, sys: 28.1 s, total: 5min 45s
Wall time: 5min 48s


#### `sorting -> head`

In [27]:
%%time
%%memit
result_np = t20_np["EDEC_TRAMT"].sort_values(ascending=False).head(5)

peak memory: 33684.78 MiB, increment: 6997.25 MiB
CPU times: user 18.8 s, sys: 3.22 s, total: 22 s
Wall time: 22.1 s


#### `nlargest`

In [28]:
%%time
%%memit
result_np = t20_np["EDEC_TRAMT"].nlargest(5)

peak memory: 33743.02 MiB, increment: 3523.82 MiB
CPU times: user 1.58 s, sys: 1.15 s, total: 2.73 s
Wall time: 2.87 s


### 2.3 건강위험지수 5 이상인 사람들의 진료내역 추출
filtering operation의 성능
- list comprehension
- apply
- pd isin
- pd query
- np isin
- pd merge
- pa ???

(+) pyarrow 데이터 타입을 지정했을 때 변화

In [29]:
gj_np.head()

,IDV_ID,SEX,AGE_GROUP,HEIGHT,WEIGHT,BLDS,TOT_CHOLE,HMG,OLIG_PROTE_CD,SGOT_AST,SGPT_ALT,SMK_STAT_TYPE_CD,DRK_YN
0,725578,2,1,140.0,45.0,91.0,216.0,13.4,1.0,20.0,11.0,1,1
1,118183,2,1,145.0,35.0,75.0,162.0,13.0,1.0,26.0,12.0,1,1
2,667818,2,1,145.0,35.0,85.0,149.0,12.7,1.0,16.0,15.0,1,1
3,877233,2,1,145.0,40.0,88.0,170.0,12.6,1.0,17.0,10.0,1,1
4,917332,2,1,145.0,40.0,85.0,242.0,12.2,1.0,9.0,10.0,1,1


In [30]:
PATIENTS = gj_np[scores_np >= 5]["IDV_ID"].unique().tolist()
print(f"Number of patients scoring above 5 : {len(PATIENTS):,}")

Number of patients scoring above 5 : 940


#### List Comprehension

In [31]:
%%time
%%memit
tmp = t20_np[[x in PATIENTS for x in t20_np["IDV_ID"]]][["IDV_ID"]]

peak memory: 37716.23 MiB, increment: 9433.26 MiB
CPU times: user 29min 49s, sys: 11.5 s, total: 30min 1s
Wall time: 30min 1s


#### `pd.DataFrame.isin()`

In [32]:
%%time
tmp = t20_np[t20_np.isin({"IDV_ID": PATIENTS})["IDV_ID"]][["IDV_ID"]]

CPU times: user 40.5 s, sys: 8.04 s, total: 48.5 s
Wall time: 48.5 s


#### `pd.DataFrame.query()`

In [33]:
%%time
tmp = t20_np[["IDV_ID"]].query("IDV_ID in @PATIENTS")

CPU times: user 21.6 s, sys: 2.86 s, total: 24.5 s
Wall time: 24.5 s


#### `pd.DataFrame.apply()`

In [34]:
%%time
tmp = t20_np[t20_np["IDV_ID"].apply(lambda x: x in PATIENTS)][["IDV_ID"]]

CPU times: user 29min 45s, sys: 9.91 s, total: 29min 55s
Wall time: 2h 16min 33s


#### `np.isin()`

In [35]:
%%time
tmp = t20_np[np.isin(t20_np["IDV_ID"].values, PATIENTS)][["IDV_ID"]]

CPU times: user 19.7 s, sys: 1.51 s, total: 21.2 s
Wall time: 21.2 s


#### `pd.DataFrame.merge()`


In [36]:
%%time
tmp = t20_np.merge(pd.Series(PATIENTS, name='IDV_ID'), how='inner', on='IDV_ID')

CPU times: user 6.37 s, sys: 1.48 s, total: 7.86 s
Wall time: 7.86 s
